In [1]:
import glob
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, to_date, input_file_name

load_dotenv("/home/jovyan/work/.env")

def main():
    
    spark = SparkSession.builder.appName("csv_to_parquet_bronze_order_details").getOrCreate()

    raw_path = os.getenv("RAW_PATH")
    bronze_path = os.getenv("BRONZE_PATH")
    
    # Path con wildcard para lecturas masivas
    search_pattern = raw_path + "/**/*orders_detail*.csv" # patron para buscar todos los archivos con nombre order_header
    
    #Buscar archivos que coincidan con el patrón
    files = glob.glob(search_pattern, recursive=True)
    
    if len(files) == 0:
        raise FileNotFoundError(
            f"[ERROR] No se encontraron archivos CSV que coincidan con el patrón: {csv_path}"
        )
    
    print(f"Archivos encontrados: {files}")
    
    # Leer csv como un dataframe
    df_raw = spark.read.csv(search_pattern, header=True, inferSchema=True)
    
    # Agregar columnas de control
    df_bronze = (
        df_raw
        .withColumn("ingestion_timestamp",   current_timestamp())       # timestamp exacto
        .withColumn("source_file",    input_file_name())         # ruta del archivo origen
    )
    
    # Escritura a bronze como parquet
    parquet_path = os.path.join(bronze_path, "orders_detail")  # relative path
    
    df_bronze.write.parquet(parquet_path, mode= 'overwrite')
    
    print(f"Bronze escrito en: {parquet_path}")
    print(f"Total Filas ingestadas: {df_bronze.count()}")

In [2]:
try:
    main()
except Exception as e:
    print(f"Error: {e}")


Archivos encontrados: ['/home/jovyan/work/data/raw/orders_detail.csv', '/home/jovyan/work/data/raw/orders_detail/orders_detail_part1.csv', '/home/jovyan/work/data/raw/orders_detail/orders_detail_part2.csv', '/home/jovyan/work/data/raw/orders_detail/orders_detail_part3.csv']
Bronze escrito en: /home/jovyan/work/data/bronze/orders_detail
Total Filas ingestadas: 11241634
